# `TEANAPS` PLAYSTORE REVIEW SCRAPPER
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

> **\*\*\* 주의사항 \*\*\***  
본 자료에서 설명하는 웹크롤링하는 방법은 해당 기법에 대한 이해를 돕고자하는 교육과 이를 활용한 연구 목적으로 사용되었으며, 대량의 무단 크롤링 및 상업적 활용을 금합니다.

## 1. 데이터 수집 준비하기

### 1-1. Selenium 라이브러리 설치

In [1]:
# 가상의 브라우저를 컨트롤 할 수 있도록 도와주는 selenium 패키지를 설치합니다.
# 아래 주석을 해지하고 셀을 실행합니다.
# 설치는 한번만 수행하면 되며, 재설치시 Requirement already satisfied: ~ 라는 메시지가 출력됩니다.

#!pip install selenium

### 1-2. Chrome Driver 다운로드 및 경로설정

- Chrome Driver 다운로드 URL : http://chromedriver.chromium.org/downloads

In [2]:
# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

# Windows 운영체제
#DRIVER_PATH = "chromedriver.exe"

# MAC, Linux 운영체제
# - 경고메시지 출력 시 보안 및 개인정보보호 메뉴에서 "확인없이 허용" 클릭 필요함
DRIVER_PATH = "./chromedriver"

### 1-3. 라이브러리 Import 및 Chrome Driver 실행

In [3]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

In [4]:
# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
# Chrome Driver를 호출합니다.
driver = webdriver.Chrome(executable_path = DRIVER_PATH, options=chrome_options)

## 2. PlayStore 리뷰 수집하기

---

### 2-1. 리뷰 URL 정보 입력

In [5]:
# 수집할 리뷰 페이지 URL 정보를 입력합니다.
URL = "https://play.google.com/store/apps/details?id=com.coupang.mobile.play&showAllReviews=true"

# 스크롤을 내릴 횟수를 입력합니다.
# 스크롤을 여러번 내릴수록 더 많은 리뷰가 수집됩니다.
SCROLL_COUNT = 5

# 리뷰를 저장할 파일명
REVIEW_FILENAME = "playstore_review.txt"

### 2-2. 리뷰 스크롤 내리기

In [6]:
driver.get(URL)

# 스크롤이 동작하지 않는 경우에만 선택적으로 적용합니다.
# 크롬창을 최대화합니다.
#driver.maximize_window()

for i in range(SCROLL_COUNT):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    print(i, "스크롤 내리기")
    time.sleep(3)
    
    #class="RveJvd snByac"
    #//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span
    
    more_button_count = len(driver.find_elements(By.CLASS_NAME,"RveJvd"))
    if more_button_count > 0:
        print(i, "[더보기] 버튼 클릭")
        driver.execute_script('window.scrollTo(0, 0);')
        driver.find_element(By.CLASS_NAME,"RveJvd").click()
    
    #try: 
    #    driver.find_element(By.CLASS_NAME,"RveJvd").click()
    #    print(i, "[더보기] 버튼 클릭")
    #except:
    #    continue

# 스크롤을 다시 맨 위로 이동
driver.execute_script('window.scrollTo(0, 0);')

review_box_list = driver.find_elements(By.CLASS_NAME, "d15Mdf")
print("> 리뷰개수 :", len(review_box_list))

0 스크롤 내리기
1 스크롤 내리기
2 스크롤 내리기
3 스크롤 내리기
3 [더보기] 버튼 클릭
4 스크롤 내리기
> 리뷰개수 : 240


### 2-3. 리뷰 수집

In [7]:
f = open(REVIEW_FILENAME, "w", encoding = "utf-8")

progress = 0 
for review_box in review_box_list:
    progress += 1
    print(progress, end="\r")
    
    # 전체보기 버튼 클릭(리뷰가 긴 경우 전체보기)
    more_button_list = review_box.find_elements(By.CLASS_NAME, "OzU4dc")
    if len(more_button_list) > 0:
        more_button_list[0].click()
    
    # 별점
    review_star = review_box.find_element(By.CLASS_NAME, "pf5lIe")
    div_tag = review_star.find_element(By.TAG_NAME, "div")
        
    star_label = div_tag.get_attribute("aria-label")
    # 작성일자
    review_date = review_box.find_element(By.CLASS_NAME, "p2TkOb").text
    # 리뷰 텍스트
    review_text = review_box.find_element(By.CLASS_NAME, "UD7Dzf").text
    
    # 파일에 기록하기 + 출력하기
    f.write(review_date + "\t" + star_label + "\t" + review_text + "\n")
    print(review_date + "\t" + star_label + "\t" + review_text)
    
f.close()

2021년 7월 8일	별표 5개 만점에 2개를 받았습니다.	여기 리뷰에 귀를 기울이셔야할 거 같아요. 와이파이가 잘 연결되어 있음에도 불구하고 자주 끊기고, 그로 인해 1080p로 설정해놓아도 화질이 좋지 않네요. 차라리 돈 더 주고 좋은 곳 가입할래요. 그리고 음량과 화면 밝기 조절하는 거 클릭이 잘 안 되어서 불편해요. 좋은 방법으로 바꿔주세요. 그리고 왜 다른 기기에서 볼 수 없도록 해놓은 건지...? 이 작은 핸드폰으로 몇 시간동안 영상을 보려니 집중도도 떨어지고 힘들어요. 이게 진짜 이해가 안 돼요. 그냥... 쿠팡 플레이는 쿠팡이 자랑 삼으려고 만든 거 같은 기분이에요.
2021년 7월 9일	별표 5개 만점에 4개를 받았습니다.	런닝맨 1박2일 같은 지상파인기예능은 에피소드가 몇개없음..가격을생각하면 괜찮지만 타 ott랑 비교하면 절대 다른걸 안보고 이거만 본다라는 선택지는없다고 봐도무방.. 키즈교육용으로는 나름 컨텐츠가 있긴함.. 아직까지도 쿠팡의 부가서비스중 하나로 봐야지 그 이상을 기대하지못함;;
2021년 7월 8일	별표 5개 만점에 1개를 받았습니다.	잘 나오다가 오류나서 페이지 안 뜬다고 꺼져버리고 다시 틀면 처음부터 재생되고. 다시 눌러도 오류로 나오지를 않아서 볼 수가 없습니다. 몇 번 다시 틀다가 안 되면 몇 시간이든 며칠이든 텀 두고 보게 되니까 재미도 떨어지고. 드라마 후반부로 가니까 더 심해서 13편을 지금 며칠 째 보고있어서 짜증나서 리뷰 쓰러 왔습니다. 오류 뭔데 이렇게 심합니까? 보라는 거야 말라는 거야. 개선은 하고 계십니까?
2021년 7월 9일	별표 5개 만점에 3개를 받았습니다.	1. 배속 재생이 안되네요? 2. 코로나 알림 뜨고 앱이 백그라운드로 재생되는데 앱 전체닫기해서 종료가 안되요; 3. 다시 플레이시 마지막 장면 저장 안되서 수동으로 이동하네요
2021년 7월 6일	별표 5개 만점에 1개를 받았습니다.	렉이 너무 많이 걸리는데요; 영상 하나 보려면 이틀 동안 누르고 있어야 하나요? 인내심 테스트인가요? 개선

2021년 6월 26일	별표 5개 만점에 4개를 받았습니다.	볼 거리들 많아져서 좋은데 제발 재생 속도 조절 가능 기능 추가해주세요ㅠㅠㅠㅠㅠ
2021년 6월 22일	별표 5개 만점에 1개를 받았습니다.	티비에 앱 다운받아서 보는데 자꾸 끊겨요. 1시간도 안돼 5번 끊김 Shaka-1001 오류 메시지 뜨면서ㅡㅡ
2021년 6월 22일	별표 5개 만점에 2개를 받았습니다.	저렴하고 컨텐츠 맘에 들고 다 좋아요. 한글자막밖에 없는건 좀 아쉽지만. 그런데 제발 재생 위치 초기화 되는것만 좀 해결해줬으면 좋겠어요ㅠ 아니 16화 보다가 종료하고 다음에 들어갔을때 15화로 되돌아가는건 좀 너무한거 아닌가요ㅠ 정확한 재생 정보 기억이 어렵다면 차라리 끄기전에 저장할 수 있게 "여기까지 시청" 같은 버튼을 넣어주세요;; 자동으로 저장될 거 처럼 동작하면서 계속 엉뚱한 에피소드를 플레이하니 환장하겠네요. 그리고 키즈프로필은 어째 삭제해도 계속 생기는건지, 또 여럿이 이용할 수 있다고 하는데 막상 쿠팡 앱부터 아이디가 다르면 로그인이 안되더라구요. 그러면 결국 다중이용 안되는거 아닌가요;; 아무튼 다른건 그렇다치는데 제발 재생위치기억은, 꼭 좀 해결해줬으면 합니다;;
2021년 7월 6일	별표 5개 만점에 1개를 받았습니다.	한시간보는데 열번 넘게 끊어지고. 이어보기도 안되고..
2021년 7월 2일	별표 5개 만점에 3개를 받았습니다.	볼만은 한데 가끔 끈기면서 안나온다 드라마나 영화 중간에 로딩이 겁나 심하게 생긴다특정 회차나 영화가 그렇다
2021년 6월 10일	별표 5개 만점에 2개를 받았습니다.	원래 잘 안남기는데 심각해서 남깁니다 싸고좋은데,, 이거 솔직히 너무 한거아닌가요 싸다고 이렇게 영화가 끊어지면서 보게 하면 거의 안보는게 낫죠.. 대사가 안들릴만큼 장면이 계속 끊기는데 어떻게 보나요.. 지웠다가 깔아도 마찬가지에요.. 예를 들어 대박부동산 보면 오 오 인이인/범 으ㄴ... 한숨을 쉬면 하 하하/아/아/아
2021년 7월 3일	별표 5개 만점에 1개를 받았습니

2021년 6월 9일	별표 5개 만점에 1개를 받았습니다.	검색 기능이 병맛!!!! 싼맛에 써본다??? 컨텐츠도 중요하지만, 기본은 지켜야 하지않을까? 빌게쵸 양반이나, 잡스 형님 같은 분들이 새로운 사용자 경험이 중요하다고 말하긴 했는데, 그 경험이 병맛 체험이었다면 지금의 MS나 애플이 존재했을까? 솔직히 최근에 여기저기서 병맛 선사하는 앱을 자주 접하다보니 "또?"라는 말이 절로 나온다. 제발 잘좀 합시다. 돈주고 알파 테스트 하고픈 사용자는 없으니까! 솔직히 지금같아서는 무료 끝나고 유료전환을 안할것 같은데....
2021년 6월 3일	별표 5개 만점에 1개를 받았습니다.	와우 사용중인데 무료라길래 사용해 봤습니다. 일단 제일큰 문제는 티비에서 시청시 드라마 1편을 보려면 한 20번 정도는 튕겨서 재생이 멈춤니다. 이걸 쿠팡에서 왜 만든건지 의문이 크게 드네요 결국 드라마 하나 보려다 포기하게 만드는 멋진 앱인거 같아요 서버문제인지 ott 개발자 문제인지 멋지네요 쿠팡 ㅎㅎ 로켓배송에나 그냥 힘써주십쇼
2021년 6월 15일	별표 5개 만점에 1개를 받았습니다.	일단 들어가려면 알림 동의함에 설정해야 들어가지고요 ㅋㅋㅋ (어이가없어서) 영상 플레이가 안됩니다 똑같은 부분을 세번이나봤는데 계속 똑같은 장면에서 멈춥니다 앱 삭제하러왔다가 평가남깁니다
2021년 6월 17일	별표 5개 만점에 5개를 받았습니다.	굳굳 근데 어딜가도 이런가격에 이런서비스 없음. 초반이라서 개선될게 있는건 당연한거임.. 앞으로 점점더 좋은퀄리티로 나아갈수있기를 응원합니다.
2021년 5월 7일	별표 5개 만점에 4개를 받았습니다.	가격이 깡패니 그냥 써요 다만 전체 화면 안되는거 제스처 안되는거 종류나 영화수 부족.pc버전 부족 다른건 다 이해하겠는데 정착 쿠팡플레이에서 약속한 건 지켜야되지 않을까요 ? pc 버전은 근방 나올것처럼 이야기 나왓는데 아직도 안나온걸 보니..좀 그러네요
2021년 4월 30일	별표 5개 만점에 5개를 받았습니다.	쿠팡 이용자 로써 너무나 만족 스러워

2021년 5월 11일	별표 5개 만점에 3개를 받았습니다.	아프리카티비, 와챠도 구글티비(크롬캐스트4세대)에 연동되서 폰이나 테블릿 없이 바로바로 볼 수 있는데, 쿠팡플레이는 핸드폰으로 꼭 따로 연결해야만 해서 많이 아쉽습니다.
2021년 6월 2일	별표 5개 만점에 3개를 받았습니다.	빔프로젝터 mitv스틱으로 보는데 폰에서 미러링연결로 봐야하니 불편해요 자꾸 끊겨서 그래도 펜트하우스 첫편 볼수있어서 행복합니다 시즌2부터 본방사수하고 그랬어서 암튼 개선해주세요 폰연결없이 넷플릭스처럼 앱다운로드되서 미러링없이볼수있길요~
2021년 4월 27일	별표 5개 만점에 4개를 받았습니다.	JTBC TV프로그램도 적극 반영해주셨으면합니다. 아는형님 재밌게 보는데 없는게 너무 아쉬워요~~ 앞으로 가성비 좋은 쿠팡플레이로 꾸준히 업그레이드되었으면합니다!!
2021년 5월 12일	별표 5개 만점에 4개를 받았습니다.	한 달 2,900원에 로켓배송에 다양한 콘텐츠 볼 수 있어서 좋네요. 왓챠랑 웨이브 쓰고 있어서 솔직히 볼 거 없지만 OTT 안 쓰시는분들한테는 나름 쓸만한 거 같아요. 뭐 나중에 시장점유율 높아지면 가격도 자연스레 오르긴 할테지만 그때는 더 다양한 컨텐츠가 있어야 할 거 같네요ㅋㅋ
2021년 5월 2일	별표 5개 만점에 1개를 받았습니다.	갤럭시탭에서 오늘 쿠팡플레이를 다운받아 설치해 본 결과 인터넷 속도는 괜찮은데, 영상과 음성이 따로 나옵니다. 이부분이 빨리 개선되어야 할 것 같습니다.
2021년 5월 2일	별표 5개 만점에 5개를 받았습니다.	대체 쿠팡플레이 왜 까는거임..? 컨텐츠량 엄청나고 화질도 깔끔. 자막 폰트 크기가 쫌 많이 크긴하지만 버전도 얼마되지 않았으니 앞으로 충분히 개선될 일. 쿠팡플레이 쓰고 싶어서라도 로켓회원 계속 할 예정
2021년 4월 28일	별표 5개 만점에 5개를 받았습니다.	쿠팡와우에 보너스로 컨텐츠 시청도 가능하니 좋네요. 컨텐츠가 많아지길 바랍니다. 구글TV도 지원했으면 좋겠습니다.
2021년 5월 4일	별표 5개 만점에

2021년 6월 5일	별표 5개 만점에 1개를 받았습니다.	별하나도 아깝네요. 쿠팡에 자꾸 떠서 들어갔더니 겉보기만 멀쩡하고 1분짜리 파닉스도 페이지를 찾을수 없다고 뜨고 재생 안되요
2021년 6월 13일	별표 5개 만점에 3개를 받았습니다.	싸니까 이해하는데 휴대폰 시청시 화면 늘리기라도 되서 전체화면으로 볼수있게라도..
2021년 5월 10일	별표 5개 만점에 5개를 받았습니다.	이용해보니 볼만한것도 많고 넘 좋은데요? 쿠팡 와우회원 연동인게 진짜 좋네요~ 영상 재생중 앞으로 10초 뒤로 10초 이런거 있음 좋겠어요~
2021년 7월 9일	별표 5개 만점에 5개를 받았습니다.	조아요..활용을 잘하고있씀니다 정말조아요 프로그램자체도 다양하여 조아요 감사드림니다..
2021년 4월 29일	별표 5개 만점에 5개를 받았습니다.	삼성TV UN65KS8000 시리즈(2016년 제품)는 지원계획에 없나요? 다른 사람들처럼 TV APP 으로 편하게 보고 싶습니다.!! 지원 부탁드립니다.!!!!!!!!!!!!!!!!!!
2021년 6월 9일	별표 5개 만점에 1개를 받았습니다.	개 쓰래기 어플. pc버전도 없이 축구독점해서 컴퓨터로 볼 수가 없음. 이번에만 가입해서 보고. 다시는 대가리 깨져도 가입할일 없을듯


In [8]:
driver.close()